# Peer-graded Assignment: Capstone Project - The Battle of Neighborhoods (Week 1)

In order to solve bussines problem that I explained in first section I will use following data. Data that will be used are retreated using Foursquare API. As free account is limited to 100, I made two with starting location on main square of each city and radius of 1500 meters which is approximately maximum walking distance for tourists and visitors (main target group of the company) and in same time that are covers most of points of interest in city centres. 

On the beggining we will impoert libraries that I plan to use later:

In [ ]:
import urllib.request
from bs4 import BeautifulSoup
import requests
from geopy.geocoders import Nominatim
import numpy as np
import pandas as pd
from scipy import stats
from scipy.interpolate import griddata
from sklearn import preprocessing 
from sklearn.model_selection import train_test_split
from sklearn import svm
from sklearn import linear_model
from sklearn.metrics import r2_score
import matplotlib.pyplot as plt 
import matplotlib.cm as cm
import matplotlib.colors as colors
from mpl_toolkits.mplot3d import Axes3D
import seaborn as sns
!pip install folium

Call for list of Wine Bars in Belgrade, Serbia:

In [4]:
CLIENT_ID = ' ' # Deleted not to be posteb public on Git
CLIENT_SECRET = ' ' # Deleted not to be posteb public on Git
VERSION = '20180605' 
LIMIT = 100 
RADIUS = 1500
CATEGORY = '4bf58dd8d48988d123941735' 
LL_LIST = ['44.8163, 20.4605']

venues_list = []              

for ll in LL_LIST:
  # create the API request URL for venues of category 'CATEGORY'
  url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&v={}&ll={}&radius={}&categoryId={}&limit={}&intent=browse'.format(
  CLIENT_ID, 
  CLIENT_SECRET,
  VERSION,   
  ll,  
  RADIUS,
  CATEGORY, 
  LIMIT)

  # make the GET request
  results = requests.get(url).json()["response"]['venues']

  # append the relevant results to venues_list
  venues_list.append([(  
  v['name'], 
  v['id'],
  v['location']['lat'], 
  v['location']['lng']) for v in results])


In [6]:
barsbelgrade = pd.DataFrame([item for venues_list in venues_list for item in venues_list])
barsbelgrade.columns = ['Name','id','Latitude','Longitude',]

barsbelgrade = barsbelgrade.drop_duplicates()
barsbelgrade

,Name,id,Latitude,Longitude
0,Ђура | Đura,548c8507498e6b13ea13b45a,44.819442,20.473780
1,Wine Art Podrum,4d448c04c3e5f04d7cae9b20,44.820772,20.458939
2,Vin Et Baguette,5bc8f5f7d807ee002c659d18,44.803471,20.461351
3,Vinarija,4e6961f71fc7442858a85ad2,44.816037,20.450477
4,Bistro Dorćol,5a451abb1b0ea523e118f16c,44.820397,20.457380
5,Delice,5f2c27ddf745981caabe3094,44.820830,20.461071
6,SirO'vina,4ec643a102d5888e9a5f20ab,44.822526,20.455601
7,Zlatni bokal,4d7a9d947ea26ea8623ec78f,44.817269,20.463888
8,Vinoteka Savamala,5617b8ce498eb55f3bb49146,44.813701,20.451764
9,Mala Toscana,5172df74e4b00121f01a01ba,44.812382,20.463953


Call for list of wine bars in Zagreb, Croatia:

In [7]:
CLIENT_ID = ' ' # Deleted not to be posteb public on Git
CLIENT_SECRET = '' # Deleted not to be posteb public on Git
VERSION = '20180605'
LIMIT = 100 
RADIUS = 1500
CATEGORY = '4bf58dd8d48988d123941735' 
LL_LIST = ['45.8129, 15.9772']

venues_list = []              

for ll in LL_LIST:
  # create the API request URL for venues of category 'CATEGORY'
  url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&v={}&ll={}&radius={}&categoryId={}&limit={}&intent=browse'.format(
  CLIENT_ID, 
  CLIENT_SECRET,
  VERSION,   
  ll,  
  RADIUS,
  CATEGORY, 
  LIMIT)

  # make the GET request
  results = requests.get(url).json()["response"]['venues']

  # append the relevant results to venues_list
  venues_list.append([(  
  v['name'], 
  v['id'],
  v['location']['lat'], 
  v['location']['lng']) for v in results])

In [8]:
barszagreb = pd.DataFrame([item for venues_list in venues_list for item in venues_list])
barszagreb.columns = ['Name','id','Latitude','Longitude',]

barszagreb = barszagreb.drop_duplicates()
barszagreb

,Name,id,Latitude,Longitude
0,La Bodega,5221c8bf11d295d4ca042f9d,45.812254,15.975071
1,Cosmopolitan Caffe&Wine bar,51694a30e4b065db5408cc28,45.812048,15.976224
2,Cheese bar,515034a7e4b069b918dc463f,45.813284,15.979173
3,Roots juice & cocktail bar,5cc5a1b664c8e1002cdedd5e,45.813827,15.973064
4,Pod starim krovovima,4d774db7fbbc224b0c700232,45.817420,15.974119
5,Bornstein Vinoteka,4bf2c16c9319a593f94dcb51,45.817866,15.978046
6,Wine Bar Karijola,4fc64954e4b0a6771dfeb48e,45.814351,15.984840
7,Pizzeria Karijola,4f922320e4b0997b9d2a5eba,45.814301,15.984910
8,"Mojo bar wine, rakia & co.",4fe83745e4b0f0c8f3d793a9,45.812225,15.984393
9,Corner bar,5394cea7498e8a555ae7d332,45.812930,15.979440
